In [1]:
trend_scan_Nsteps = 50
measure_time=1
measure_board_list =  [ "0902", "0905"]  



from matplotlib import pyplot as plt
import numpy as np
import pasttrec_ctrl as ptc
import json
import sigproc_kit
import tdc_daq as td
import baseline_calib
import ROOT
import db
from cw_pasttrec_functions import *

from my_utils import *

set_width_max_of_jupyterpad()

scaler_rates = td.scaler_rate_of_board(measure_board_list[0],1) 
average_scaler_list        =  [[ []  for i in range(trend_scan_Nsteps) ] for i in range(len(measure_board_list)) ]
average_scaler_list_err    =  [[ []  for i in range(trend_scan_Nsteps) ] for i in range(len(measure_board_list)) ]
#ptc.init_active_boards(pt_pktime,pt_gain,pt_threshold_default) 
# get scaler rates for chosen threshold/gain/peaking time as simple estimate of noise:
scaler_list_trend        =  [ []  for i in range(trend_scan_Nsteps) ] 

scaler_list_trend_channel = [ []  for i in range(16)]
for b in range(0,len(measure_board_list)):
 name = measure_board_list[b]
 for p in range(0,trend_scan_Nsteps):
            scaler_rates = td.scaler_rate_of_board(name,measure_time) 
            #scaler_rates = td.scaler_rate("0x0350",[48,49],1)

            print(scaler_rates)
            if any(i > 1E7 for i in scaler_rates):
                #scaler_list_trend += [ scaler_rates ]
                scaler_list = scaler_rates
                
                plt.errorbar(range(0,len(scaler_list)),scaler_list, yerr=None, xerr=None, fmt='o:', alpha=0.4,label = "channel {:d}".format(p))
                plt.xlabel("trend ")
                plt.ylabel("scaler rates")  
            for ch in range(0,len(scaler_rates)):
               if  scaler_rates[ch] < 1E7 :
                scaler_list_trend_channel[ch] += [ scaler_rates[ch] ]
                
 plt.show()
 plt.legend()

 for ch in range(0,len(scaler_rates)):
    if scaler_list_trend_channel[ch]:
            plt.errorbar(range(0,len(scaler_list_trend_channel[ch])),scaler_list_trend_channel[ch], yerr=None, xerr=None, fmt='o:', alpha=0.9,label = "channel {:d}".format(ch))
            plt.xlabel("trend ")
            plt.ylabel("scaler rates")  
            plt.show()

Welcome to JupyROOT 6.14/04


[10287.0, 10947.0, 13459.0, 13947.0, 4232.0, 12884.0, 242441.0, 14780.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[87801.0, 133177.0, 178967.0, 206776.0, 193636.0, 165078.0, 319323.0, 76147.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[37561.0, 56068.0, 77512.0, 91716.0, 81214.0, 72200.0, 272028.0, 37662.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[39108.0, 58393.0, 80770.0, 94604.0, 82455.0, 74416.0, 271410.0, 38385.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
[44947.0, 67947.0, 92754.0, 107993.0, 96089.0, 83866.0, 277726.0, 41696.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[52562.0, 79879.0, 108157.0, 125947.0, 112402.0, 96001.0, 286009.0, 45650.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[58707.0, 88757.0, 120377.0, 139283.0, 125835.0, 106684.0, 292817.0, 50553.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0]
[62400.0, 95416.0, 130676.0, 151267.0, 137928.0, 116145.0, 293965.0, 53548.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]
[65722.0, 99942.0, 136705.0, 161019.0, 147918.0, 125038.0, 298726.0, 570

No handles with labels found to put in legend.


[2488.0, 2320.0, 2152.0, 2727.0, 1067.0, 793.0, 1668.0, 1739.0, 4593.0, 5009.0, 2532.0, 1146.0, 3052.0, 4042.0, 2349.0, 1501.0]
[1509.0, 1406.0, 1285.0, 1671.0, 544.0, 421.0, 989.0, 1041.0, 3199.0, 3630.0, 1750.0, 685.0, 2261.0, 2987.0, 1677.0, 991.0]
[1740.0, 1614.0, 1501.0, 1891.0, 687.0, 528.0, 1174.0, 1245.0, 3427.0, 3866.0, 1948.0, 814.0, 2347.0, 3157.0, 1813.0, 1150.0]
[5779.0, 2425.0, 3946.0, 6233.0, 10286.0, 2354.0, 1949.0, 14193.0, 12053.0, 28886.0, 2829.0, 2202.0, 7333.0, 61309.0, 13799.0, 4718.0]
[7.0, 9.0, 4.0, 2.0, 3.0, 5.0, 4.0, 7.0, 18.0, 19.0, 6.0, 4.0, 13.0, 15.0, 3.0, 3.0]
[1134.0, 283.0, 691.0, 1184.0, 2516.0, 466.0, 239.0, 3478.0, 2508.0, 8620.0, 516.0, 635.0, 1999.0, 19611.0, 4347.0, 1467.0]
[1527.0, 2487.0, 3491.0, 3824.0, 3415.0, 2797.0, 1849.0, 1053.0, 902.0, 593.0, 410.0, 271.0, 264.0, 216.0, 195.0, 168.0]
[8353.0, 12061.0, 17293.0, 19662.0, 19561.0, 13575.0, 8733.0, 8304.0, 7763.0, 11843.0, 2240.0, 1734.0, 3126.0, 21845.0, 5488.0, 2091.0]
[35846.0, 58849.0, 81

KeyboardInterrupt: 

In [ ]:
trend_scan_Nsteps = 500
measure_time=1
# measure_board_list =  [ "0902", "0905"]  
measure_board_list =  [ "0902"]
channellist = [48,49]


from matplotlib import pyplot as plt
import numpy as np
import pasttrec_ctrl as ptc
import json
import sigproc_kit
import tdc_daq as td
import baseline_calib
import ROOT
import db
from cw_pasttrec_functions import *

from my_utils import *

set_width_max_of_jupyterpad()

scaler_rates = td.scaler_rate_of_board(measure_board_list[0],1) 
average_scaler_list        =  [[ []  for i in range(trend_scan_Nsteps) ] for i in range(len(measure_board_list)) ]
average_scaler_list_err    =  [[ []  for i in range(trend_scan_Nsteps) ] for i in range(len(measure_board_list)) ]
#ptc.init_active_boards(pt_pktime,pt_gain,pt_threshold_default) 
# get scaler rates for chosen threshold/gain/peaking time as simple estimate of noise:
scaler_list_trend        =  [ []  for i in range(trend_scan_Nsteps) ] 

scaler_list_trend_channel = [ []  for i in range(16)]
for b in range(0,len(measure_board_list)):
 name = measure_board_list[b]
 for p in range(0,trend_scan_Nsteps):
#             scaler_rates = td.scaler_rate_of_board(name,measure_time) 
            scaler_rates = td.scaler_rate("0x0350",channellist,1)

            print(scaler_rates)
 
            for ch in range(0,len(scaler_rates)):
               if  scaler_rates[ch] < 1E7 :
                scaler_list_trend_channel[ch] += [ scaler_rates[ch] ]



for ch in range(0,len(scaler_rates)):
                if scaler_list_trend_channel[ch]:
                        plt.errorbar(range(0,len(scaler_list_trend_channel[ch])),scaler_list_trend_channel[ch], yerr=None, xerr=None, fmt='o:', alpha=0.9,label = "channel {:d}".format(channellist[ch]))
                        plt.xlabel("trend ")
                        plt.ylabel("scaler rates")  
                        plt.legend()
                        plt.show()
